<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/update_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")
install_package("xlrd>=2.0.1")
install_package("openpyxl")

In [4]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas

# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

#Players Update

In [ ]:
list_urls=get_file_content('data/list_all_urls.txt')
list_player_names=get_file_content('data/list_all_players_names.txt')
fixed_table=get_file_content('data/df_player_fixed.csv')

url_ranking='https://www.atptour.com/en/rankings/singles?rankRange=1-200'

list_player_names_new,list_urls_new=players_from_url(url_ranking)
list_player_names_new=[k for k in list_player_names_new if k not in list_player_names]
list_urls_new=[k for k in list_urls_new if k not in list_urls]

df_new=create_new_fixed_table('', list_player_names_new, list_urls_new)

In [ ]:
import pandas as pd

df_fixed_all=pd.concat([fixed_table,df_new])
list_player_names_all=list_player_names_new+list_player_names
list_urls_all=list_urls_new+list_urls

add_file_github('data/list_all_urls.txt',str(list_urls_all))
add_file_github('data/list_all_players_names.txt',str(list_player_names_all))
add_file_github('data/df_player_fixed.csv',df_fixed_all.to_csv(index=False))


# Games Update

In [ ]:
import datetime

# Get the current date
current_date = datetime.date.today()
previous_week = current_date - datetime.timedelta(days=(7))

week_url=find_list_weekly_ranking_url(previous_week,current_date,nb_players_ranking=300)[0]
ranking_week=get_ranking(week_url)

previous_ranking=get_file_content("data/df_all_ranking.csv")
df_ranking_all=pd.concat([previous_ranking,ranking_week]).reset_index(drop=True).drop_duplicates()
add_file_github("data/df_all_ranking.csv",df_ranking_all.to_csv(index=False))


# Odds Update

We use a brut force methodolodgy here, recreating the table from scratch every week.

In [ ]:
# URL of the online-hosted Excel dataset
columns_selected=['Location','Tournament','Date','Winner','Loser']
df_all_betting=pd.DataFrame(columns=columns_selected)
for year in [2001+k for k in range(23)]:
  excel_url = "http://www.tennis-data.co.uk/"+str(year)+"/"+str(year)+".xlsx"

  # Read the Excel dataset into a Pandas DataFrame
  df = pd.read_excel(excel_url)

  odds_columns=[k for k in df.columns if k[-1]=='L' or k[-1]=='W']
  columns_selected_year=columns_selected+odds_columns
  # print(df.columns)
  df=df[columns_selected_year]

  # Display the first few rows of the DataFrame to verify the data
  df_all_betting=pd.concat([df_all_betting,df]).reset_index(drop=True)

add_file_github("data/df_all_betting.csv",df_all_betting.to_csv(index=False))